In [26]:
import torch

In [28]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2

z = y * y * 3
out = z.mean()

print(x)
print(y)
print(z)
print(out)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


In [29]:
out.backward()

In [31]:
print(x.grad)

None
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
